# Leaky Integrate and Fire Neuron

Starting from a biophysically detailed model of the current impinging on a neuron's membrane:

$I = C_m \frac{dV}{dt} + g_L(V-E_L) + I_{Na} + I_{K} + I_{H} + I_{AHP}$

We simplify the ions involved, neglecting the shape of the action potential and replacing it with a threshold condition.

$\tag{1}
C_m\frac{dV}{dt}=-g_L(V-E_L)+I$
Note:
$\tag{2}
R = 1 / g_L$
$\tag{3}
\tau_m = R \cdot C_m$

$\tag{1}
\tau_m \frac{dV}{dt} = -(V - E_L) + RI$
$\tag{2}
if \quad V(t) = V_{th} \quad then \quad V(t + \Delta) = E_L$

In [ ]:
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from matplotlib import pyplot as plt

# General simulation parameters
dt = 0.02  # Numerical time step
# taux = 10  # (1,100) OU time constant
# sigma = 0.15  # (0.05,0.5) Noise scaling
v_spike = 20  # For drawing nice spikes
N = 1  # Number of neurons
# I0 = 1  # Maximum current
Tdur = 150  #  Simulation duration
Tstim = 100  # Stimulus duration
stimuli = ['step', 'delta', 'sine', 'ramp']  #  Types of stimulus

n_steps = int(Tdur/dt)
sim_times = np.linspace(0, Tdur, num=n_steps+1)



def dvdt(t, sol, E_L, g_L, C):
    v = sol #[:, 0]
    v_diff = v - E_L
    return np.array((-g_L * v_diff)/C).T


def simulate(stimulus='step', g_L=30, E_L=-70.6, V_th=-50.4, V_r=-70.6, 
             C=281, Trefract=0, I0=1, Tstim=100):  # noise=False

    s0 = int((Tdur - Tstim) / 2 / dt)
    s_end = s0 + int(Tstim / dt)
    Input = np.zeros_like(sim_times)

    if stimulus == 'step':
        Input[s0:s_end] = I0
    elif stimulus == 'delta':
        s_end = s0 + int(1 / dt)  # 1 ms
        Input[s0:s_end] = I0
    elif stimulus == 'sine':
        f = 0.5
        stim_times = np.linspace(0, (s_end-s0)*dt, num=s_end-s0) + 3*np.pi
        Input[s0:s_end] = (1 + np.sin(f * stim_times)) * I0 / 2
    elif stimulus == 'ramp':
        Input[s0:s_end] = I0 * np.linspace(0, 1, num=s_end-s0)

    v0 = E_L  # Initial membrane potential
    var0 = np.ones(N) * v0  # initial condition
    n_hold_steps = int(Trefract / dt)
    theta = V_th
    spikes = np.zeros((N, len(sim_times)), dtype=bool)
    v_new = np.zeros((N, len(sim_times)))

    # Initialise solution variables
    v_new[:, 0] = var0 #np.ones(N) * v0 #np.tile(var0, (N, 1))
    hold_steps = np.zeros(N, dtype=int)
    
    # I_scale = 1e3 / C
    I_scale = 1e3
    current = np.tile(I_scale*Input, (N, 1))

    # if noise:
    #     noise = get_noise(N, n_steps+1, dt, taux, sigma)
    # else:
    #     noise = np.zeros((N, n_steps+1))
    # scaled_noise = I_scale * noise
    
    # for i in range(1, len(sim_times)):

        # rhs = dvdt(sim_times[i], v_new[:, i-1], E_L, g_L, C)
        # rhs += current[:, i] #+ scaled_noise[:, i]  [:, 0]
        # v_new[:, i] = v_new[:, i-1] + dt * rhs

    for i in range(len(sim_times) - 1):

        v_new[:, i+1] = v_new[:, i] + dt * (-g_L * (v_new[:, i] - E_L) + current[:, i+1]) / C

        spiked = np.where(v_new[:, i+1] > theta)[0]

        if spiked.size > 0:
            v_new[spiked, i+1] = V_r
            spikes[spiked, i+1] = True  # TODO: spikes could be removed and t could be stored directly

        if n_hold_steps > 0:
            hold_steps[spiked] = n_hold_steps  # neurons that spiked will be held for n_hold_steps steps
            refractory = hold_steps > 0
            v_new[refractory, i+1] = V_r
            hold_steps -= 1  # countdown for the held neurons

    v_new[spikes] = v_spike
#     spiketimes = [sim_times[spikes[n, :]] for n in range(N)]
    
    fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(14,8), gridspec_kw={'height_ratios':[1,3]})
    axes[0].plot(sim_times, Input)
    axes[0].set_ylabel('$I$ [nA]')
    axes[1].plot(sim_times, v_new[0, :], label=rf'$\tau_m = {C/g_L:.3} ms$')
    axes[1].set_ylim(-100, 30)
    axes[1].axhline(y=theta, linestyle=':', color='grey', label='Threshold') #, xmin=0, xmax=1)
    axes[1].set_ylabel('$V$ [mV]')
    axes[1].set_xlabel('Time [ms]')
    axes[1].legend()


# simulate(stimulus='step')
interact(simulate, stimulus=stimuli, g_L=(0.,40), E_L=(-100.,-50), V_r=(-100.,-50),
         C=(5.,500), V_T=(-65.,-30), Trefract=(0.,5), I0=(0.,1), Tstim=(0.,100)) #, noise=True)